In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import utils
import random

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# hyperparameters
iterations = 100
batch_size = 32
learning_rate = 0.001
reg_eta = 0.001

# dimensionalities
dim_lstm = 300
dim_word = 300
dim_aspect = 5
dim_aspect_embedding = 300
dim_sentence = 80
dim_polarity = 3

# setup utils object
u = utils.UTILS(batch_size, dim_sentence, dim_polarity)

In [3]:
# define tf placeholders
X = tf.placeholder(tf.int32, [None, dim_sentence])
y = tf.placeholder(tf.float32, [None, dim_polarity])
seqlen = tf.placeholder(tf.int32, [None])
aspects = tf.placeholder(tf.int32, [None])

In [4]:
# define tf variables
with tf.variable_scope('aspect_embedding_vars', reuse = tf.AUTO_REUSE):
    va = tf.get_variable(
        name = 'aspect_matrix_Va',
        shape = [dim_aspect, dim_aspect_embedding],
        initializer = tf.random_normal_initializer(0, 0.003),
        regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
    )
    wv = tf.get_variable(
        name = 'aspect_Wv',
        shape = [dim_aspect_embedding, dim_aspect_embedding],
        initializer = tf.random_normal_initializer(0, 0.003),
        regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
    )
with tf.variable_scope('attention_vars', reuse = tf.AUTO_REUSE):
    wh = tf.get_variable(
        name = 'M_tanh_Wh',
        shape = [dim_lstm, dim_lstm],
        initializer = tf.random_normal_initializer(0, 0.003),
        regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
    )
    w = tf.get_variable(
        name = 'alpha_softmax_W',
        shape = [dim_lstm + dim_aspect_embedding, 1],
        initializer = tf.random_normal_initializer(0, 0.003),
        regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
    )
    wp = tf.get_variable(
        name = 'hstar_tanh_Wp',
        shape = [dim_lstm, dim_lstm],
        initializer = tf.random_normal_initializer(0, 0.003),
        regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
    )
    wx = tf.get_variable(
        name = 'hstar_tanh_Wx',
        shape = [dim_lstm, dim_lstm],
        initializer = tf.random_normal_initializer(0, 0.003),
        regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
    )
with tf.variable_scope('output_softmax_vars', reuse = tf.AUTO_REUSE):
    ws = tf.get_variable(
        name = 'y_softmax_Ws',
        shape = [dim_lstm, dim_polarity],
        initializer = tf.random_normal_initializer(0, 0.003),
        regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
    )
    bs = tf.get_variable(
        name = 'y_softmax_Bs',
        shape = [dim_polarity],
        initializer = tf.random_normal_initializer(0, 0.003),
        regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
    )    

Instructions for updating:
Use the retry module or similar alternatives.


In [5]:
# define lstm model
def dynamic_lstm(inputs, seqlen, aspects):
    inputs = tf.nn.dropout(inputs, keep_prob=1.0)
    with tf.name_scope('lstm_model'):
        # slice the corresponding vai from va
        vai = tf.gather(va, aspects) # batch_size x dim_aspect_embedding
        lstm_cell = tf.contrib.rnn.LSTMCell(dim_lstm)
        H, state = tf.nn.dynamic_rnn(
            lstm_cell,
            inputs = inputs,
            sequence_length = seqlen,
            dtype = tf.float32,
            scope = 'lstm'
        )
        size = tf.shape(H)[0]
        wv_vai = tf.matmul(vai, wv) # batch_size x dim_aspect_embedding
        # stacking Wv x Va along sentence length
        wv_vai = [wv_vai for i in range(dim_sentence)]
        wv_vai_en = tf.stack(wv_vai, axis = 1) # batch_size x dim_sentence x dim_aspect_embedding
        wv_vai_en = tf.reshape(wv_vai_en, [-1, dim_aspect_embedding]) # (batch_size * dim_sentence) x dim_aspect_embedding
        H_1 = tf.reshape(H, [-1, dim_lstm]) # (batch_size * dim_sentence) x dim_lstm
        wh_H = tf.matmul(H_1, wh) # (batch_size * dim_sentence) x dim_lstm
        # concatenate wh_H and wv_va_En for inputting to tanh
        wh_H_wv_vai_en = tf.concat([wh_H, wv_vai_en], 1) # (batch_size * dim_sentence) x (dim_lstm + dim_aspect_embedding)
        M = tf.tanh(wh_H_wv_vai_en) # (batch_size * dim_sentence) x (dim_lstm + dim_aspect_embedding)
        alpha = tf.nn.softmax(tf.matmul(M, w)) # (batch_size * dim_sentence)
        alpha = tf.reshape(alpha, [-1, 1, dim_sentence]) # batch_size x 1 x dim_sentence
        index = tf.range(0, size) * dim_sentence + seqlen - 1 # batch_size
        hn = tf.gather(tf.reshape(H, [-1, dim_lstm]), index)  # batch_size x dim_lstm
        r = tf.reshape(tf.matmul(alpha, H), [-1, dim_lstm]) # batch_size x dim_lstm
        h_star = tf.tanh(tf.matmul(r, wp) + tf.matmul(hn, wx)) # batch_size x dim_lstm
        predict = tf.matmul(h_star, ws) + bs # batch x dim_polarity
    return predict


In [6]:
# define operations
# tf.reset_default_graph()
pred = dynamic_lstm(tf.nn.embedding_lookup(u.gloveDict, X), seqlen, aspects)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = pred, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
correct = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()

/usr/local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [7]:
# full dataset training
test_X, test_y, test_seqlen, test_aspects = u.getData('test')
train_X, train_y, train_seqlen, train_aspects = u.getData('train')
with tf.Session() as sess:
    sess.run(init)
    for i in range(iterations):
        sess.run(optimizer, feed_dict = {X: train_X, y: train_y, seqlen: train_seqlen, aspects: train_aspects})
#         if i > 0 and i % 4 == 0:
        loss_train, accuracy_train = sess.run([loss, accuracy], feed_dict = {X: train_X, y: train_y, seqlen: train_seqlen, aspects: train_aspects})
        print('step: %s, train loss: %s, train accuracy: %s' % (i, loss_train, accuracy_train))
        loss_test, accuracy_test = sess.run([loss, accuracy], feed_dict = {X: test_X, y: test_y, seqlen: test_seqlen, aspects: test_aspects})
        print('step: %s, test loss: %s, test accuracy: %s' % (i, loss_test, accuracy_test))

step: 0, train loss: 1.0188078, train accuracy: 0.6196703
step: 0, test loss: 0.9933252, test accuracy: 0.6752312
step: 1, train loss: 0.94880563, train accuracy: 0.6196703
step: 1, test loss: 0.88048136, test accuracy: 0.6752312
step: 2, train loss: 0.9366695, train accuracy: 0.6196703
step: 2, test loss: 0.836337, test accuracy: 0.6752312
step: 3, train loss: 0.9341613, train accuracy: 0.6196703
step: 3, test loss: 0.8282824, test accuracy: 0.6752312
step: 4, train loss: 0.9304846, train accuracy: 0.6196703
step: 4, test loss: 0.83189875, test accuracy: 0.6752312
step: 5, train loss: 0.9276022, train accuracy: 0.6196703
step: 5, test loss: 0.8387933, test accuracy: 0.6752312
step: 6, train loss: 0.9197108, train accuracy: 0.6196703
step: 6, test loss: 0.83817595, test accuracy: 0.6752312
step: 7, train loss: 0.914806, train accuracy: 0.619386
step: 7, test loss: 0.83763844, test accuracy: 0.6752312
step: 8, train loss: 0.91689193, train accuracy: 0.619386
step: 8, test loss: 0.843333

KeyboardInterrupt: 

In [7]:
# batch training
test_X, test_y, test_seqlen, test_aspects = u.getData('test')
with tf.Session() as sess:
    sess.run(init)
    for i in range(iterations):
        batch_X, batch_y, batch_seqlen, batch_aspects = u.nextBatch(batch_size)
        sess.run(optimizer, feed_dict = {X: batch_X, y: batch_y, seqlen: batch_seqlen, aspects: batch_aspects})
        if i > 0 and i % 4 == 0:
            loss_train, accuracy_train = sess.run([loss, accuracy], feed_dict = {X: batch_X, y: batch_y, seqlen: batch_seqlen, aspects: batch_aspects})
            print('step: %s, train loss: %s, train accuracy: %s' % (i, loss_train, accuracy_train))
            loss_test, accuracy_test = sess.run([loss, accuracy], feed_dict = {X: test_X, y: test_y, seqlen: test_seqlen, aspects: test_aspects})
            print('step: %s, test loss: %s, test accuracy: %s' % (i, loss_test, accuracy_test))

step: 4, train loss: 1.0626603, train accuracy: 0.5
step: 4, test loss: 1.064084, test accuracy: 0.60328877
step: 8, train loss: 1.0036991, train accuracy: 0.59375
step: 8, test loss: 0.96842396, test accuracy: 0.6382323
step: 12, train loss: 0.97157586, train accuracy: 0.6875
step: 12, test loss: 0.929753, test accuracy: 0.6115108
step: 16, train loss: 1.0183179, train accuracy: 0.625
step: 16, test loss: 1.2224764, test accuracy: 0.5272353
step: 20, train loss: 0.9538225, train accuracy: 0.5625
step: 20, test loss: 0.9015234, test accuracy: 0.58992803
step: 24, train loss: 0.8981655, train accuracy: 0.53125
step: 24, test loss: 0.79759854, test accuracy: 0.68859196
step: 28, train loss: 0.9151036, train accuracy: 0.53125
step: 28, test loss: 0.9973002, test accuracy: 0.62487155
step: 32, train loss: 0.83227533, train accuracy: 0.59375
step: 32, test loss: 0.8038069, test accuracy: 0.663926
step: 36, train loss: 0.8009197, train accuracy: 0.59375
step: 36, test loss: 0.7305893, test a

step: 288, train loss: 0.6384232, train accuracy: 0.75
step: 288, test loss: 0.59275424, test accuracy: 0.7553957
step: 292, train loss: 0.549507, train accuracy: 0.8125
step: 292, test loss: 0.6877318, test accuracy: 0.7194245
step: 296, train loss: 0.34573045, train accuracy: 0.90625
step: 296, test loss: 0.64270097, test accuracy: 0.7451182
step: 300, train loss: 0.36133945, train accuracy: 0.90625
step: 300, test loss: 0.56728935, test accuracy: 0.7790339
step: 304, train loss: 0.7867471, train accuracy: 0.65625
step: 304, test loss: 0.77320266, test accuracy: 0.692703
step: 308, train loss: 0.3472127, train accuracy: 0.84375
step: 308, test loss: 0.6192323, test accuracy: 0.75128466
step: 312, train loss: 0.40323606, train accuracy: 0.8125
step: 312, test loss: 0.5429678, test accuracy: 0.7893114
step: 316, train loss: 0.81591547, train accuracy: 0.78125
step: 316, test loss: 0.5857284, test accuracy: 0.76875645
step: 320, train loss: 0.56856173, train accuracy: 0.8125
step: 320, 